In [2]:
import numpy as np
from itertools import islice

In [3]:
def add_blank(seq):
    seq = "_" + seq
    return seq

def initialize_scoring_matrix(sequence_a, sequence_b): 
    score_matrix = []
    for element_b in sequence_b:
        row = []
        for element_a in sequence_a:
            row.append(0)
        score_matrix.append(row)
    return score_matrix

def initialize_backpointer_matrix(sequence_a, sequence_b): 
    score_matrix = []
    for element_b in sequence_b:
        row = []
        for element_a in sequence_a:
            row.append((None, None))
        score_matrix.append(row)
    return score_matrix

def k_largest_index_argsort(a, k):
    idx = np.argsort(a.ravel())[:-k-1:-1]
    return np.column_stack(np.unravel_index(idx, a.shape))

def get_aligned_seq(indices):
    curr_ind = tuple(indices)
    next_ind = backpointer_matrix[curr_ind[0]][curr_ind[1]]
    aligned_a = ""
    aligned_b = ""

    while (curr_ind != (None, None) and curr_ind[0] > 0 and curr_ind[1] > 0):
        if (curr_ind[0] == next_ind[0]):
            aligned_b = sequence_b[0] + aligned_b
            aligned_a = sequence_a[curr_ind[1]] + aligned_a
        elif (curr_ind[1] == next_ind[1]):
            aligned_a = sequence_a[0] + aligned_a
            aligned_b = sequence_b[curr_ind[0]] + aligned_b 
        else:
            aligned_b = sequence_b[curr_ind[0]] + aligned_b 
            aligned_a = sequence_a[curr_ind[1]] + aligned_a
        curr_ind = next_ind
        
        if (curr_ind == (None, None)): break
        next_ind = backpointer_matrix[curr_ind[0]][curr_ind[1]]
    
    return aligned_a, aligned_b

def forward_propogation(sequence_a, sequence_b):
    for row_ind, row in islice(enumerate(score_matrix), 1, None):
        for col_ind, score_element in islice(enumerate(row), 1, None):
            b_char = sequence_b[row_ind]
            a_char = sequence_a[col_ind]

            if (b_char == a_char): element_score = match_score
            else: element_score = mismatch_score

            ind_score, score = argmax([score_matrix[row_ind-1][col_ind-1] + element_score,
                                       score_matrix[row_ind-1][col_ind] + gap_score,
                                       score_matrix[row_ind][col_ind-1] + gap_score, 
                                       0])

            score_matrix[row_ind][col_ind] = score

            backpointer_indices = {
                0: lambda r_ind, c_ind: (r_ind-1, c_ind-1),
                1: lambda r_ind, c_ind: (r_ind-1, c_ind),
                2: lambda r_ind, c_ind: (r_ind, c_ind-1)
            }.get(ind_score, lambda r_ind, c_ind: (None, None))(row_ind, col_ind)

            backpointer_matrix[row_ind][col_ind] = backpointer_indices
            
    return score_matrix, backpointer_matrix

argmax = lambda y: max(enumerate(y), key=lambda x: x[1])

In [4]:
sequence_a = "CGTGAATTCAT"
sequence_b = "GACTTAC"

sequence_a = add_blank(sequence_a)
sequence_b = add_blank(sequence_b)

gap_score = -4
match_score = 5
mismatch_score = -3

score_matrix = initialize_scoring_matrix(sequence_a, sequence_b)
backpointer_matrix = initialize_backpointer_matrix(sequence_a, sequence_b)

score_matrix, backpointer_matrix = forward_propogation(sequence_a, sequence_b)

largest_indices = k_largest_index_argsort(np.array(score_matrix), len(score_matrix)*len(score_matrix[0]))
largest_indices_boundary = list(filter(lambda x: x[0] == len(score_matrix) - 1 or x[1] == len(score_matrix[0]) - 1, largest_indices))

In [6]:
k = 10

for i in range(0, k):
    aligned_a, aligned_b = get_aligned_seq(largest_indices[i])
    print("Percent Match: %1.2f%%" % (sum([x == y for x, y in zip(aligned_a, aligned_b)])*100/len(aligned_a)))
    print("Score: %d" % (score_matrix[largest_indices[i][0]][largest_indices[i][1]]))
    print(aligned_a)
    print(aligned_b)
    print()

Percent Match: 71.43%
Score: 18
GAATT_C
GACTTAC

Percent Match: 71.43%
Score: 18
GAATTCA
GACTT_A

Percent Match: 80.00%
Score: 17
GAATT
GACTT

Percent Match: 62.50%
Score: 15
GAATTCAT
GACTT_AC

Percent Match: 62.50%
Score: 14
GAATTCAT
GACTT_A_

Percent Match: 62.50%
Score: 14
GAATTCA_
GACTT_AC

Percent Match: 66.67%
Score: 14
GAATTC
GACTTA

Percent Match: 66.67%
Score: 13
GAATTC
GACTT_

Percent Match: 66.67%
Score: 13
GAATT_
GACTTA

Percent Match: 75.00%
Score: 12
GAAT
GACT

